In [6]:
# Imports

# Algorithms:
from koebe.algorithms.incrementalConvexHull import incrConvexHull, orientationPointE3, randomConvexHullE3
from koebe.algorithms.hypPacker import *
from koebe.algorithms.sampling import surfaceSampling, boundarySampling
from koebe.algorithms.poissonDiskSampling import slowAmbientSurfaceSampling, slowAmbientBoundarySampling
from koebe.algorithms.cvt import weightedCVT, worldToImgPixelCoords
from koebe.algorithms.tutteEmbeddings import tutteEmbeddingE2

# Graphics:
from koebe.graphics.spherical2viewer import *
from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle, E2Viewer

# Geometries:
from koebe.geometries.orientedProjective2 import DiskOP2
from koebe.geometries.euclidean3 import PointE3, VectorE3, SegmentE3
from koebe.geometries.euclidean2 import SegmentE2, PointE2, VectorE2, CircleE2
from koebe.geometries.commonOps import inner_product31

# Linear Algebra:
import numpy as np
from numpy.linalg import matrix_rank, norm
from scipy.linalg import null_space

# Image creation:
from PIL import Image, ImageDraw, ImageFilter

# Other:
import random

In [64]:
poly = randomConvexHullE3(4)
poly.outerFace = poly.faces[0]

tutteGraph = tutteEmbeddingE2(poly)

dists = [(v.data - PointE3.O).normSq() for v in tutteGraph.verts]
closestToOriginIdx = dists.index(min(dists))
packing, _ = maximalPacking(
    tutteGraph, 
    num_passes=1000, 
    centerDartIdx = tutteGraph.darts.index(tutteGraph.verts[closestToOriginIdx].aDart)
)

# Anotate each vertex with its index in
# the vertex array:
for vIdx in range(len(packing.verts)):
    packing.verts[vIdx].index = vIdx

In [72]:
def scaleCircleE2(c, scale):
    return CircleE2(center = PointE2(c.center.x * scale, c.center.y * scale),
                    radius = c.radius * scale)

viewer = S2Viewer(800, 800)
#scale = 2
# for scale in range(1, 300):
#     diskSet = [DiskOP2.fromCircleE2(scaleCircleE2(v.data.toPoincareCircleE2(), scale / 30.0)).toDiskS2() 
#                for v in packing.verts]

#     dualPoints = [d.dualPointOP3 for d in diskSet]


#     blueStyle = makeStyle(stroke = "#00f", strokeWeight = 0.5)
#     redStyle = makeStyle(stroke = "#f00", strokeWeight = 0.5)

#     viewer.addAll(diskSet)
#     for d in diskSet:
#         viewer.setStyle(d, blueStyle)

#     viewer.addAll(dualPoints)
    
#     if scale != 29:
#         viewer.pushAnimFrame()

def perturb(disk):
    #r = (random.random() - 0.5) / (disk.d * 50)
    r = -0.5
    return DiskS2(disk.a, disk.b, disk.c, disk.d + r)

scale = 10

diskSet = [perturb(DiskOP2.fromCircleE2(scaleCircleE2(v.data.toPoincareCircleE2(), scale)).toDiskS2())
           for v in packing.verts]


dualPoints = [d.dualPointOP3.toPointE3() for d in diskSet]


blueStyle = makeStyle(stroke = "#00f", strokeWeight = 0.5)
redStyle = makeStyle(stroke = "#f00", strokeWeight = 0.5)
purpleStyle = makeStyle(stroke = "#f0f", strokeWeight = 0.5)
redFill = makeStyle(fill = "#f00")


blueStroke1 = makeStyle(stroke = "#00f", strokeWeight = 1)
redStroke1 = makeStyle(stroke = "#f00", strokeWeight = 1)
purpleStroke1 = makeStyle(stroke = "#f0f", strokeWeight = 1)

# Add the disks
viewer.addAll(diskSet)
for d in diskSet:
    viewer.setStyle(d, blueStyle)
    
viewer.addAll(dualPoints)

# Add the orthos
def ortho(face):
    return CPlaneS2.throughThreeDiskS2(*[diskSet[v.index] for v in face.vertices()])

def dualPtCP(ortho):
    return ortho.dualDiskS2.dualPointOP3.toPointE3()

for fIdx in range(len(packing.faces)):
    f = packing.faces[fIdx]
    f.index = fIdx
    f.data = ortho(f)

orthos = [f.data for f in packing.faces]
viewer.addAll(orthos)
for o in orthos:
    viewer.setStyle(o, redStyle)
    dualPt = dualPtCP(o)
    viewer.add(dualPt)
    viewer.setStyle(dualPt, redFill)

# Add the regular-regular edges and special-special edges
for e in packing.edges:
    i, j = e.aDart.origin.index, e.aDart.dest.index
    if diskSet[i].inversiveDistTo(diskSet[j]) <= 1:
        seg = SegmentE3(dualPoints[i], dualPoints[j])
        viewer.add(seg)
        viewer.setStyle(seg, blueStroke1)
    else:
        p, q = dualPtCP(e.aDart.face.data), dualPtCP(e.aDart.twin.face.data)
        seg = SegmentE3(p, q)
        viewer.add(seg)
        viewer.setStyle(seg, redStroke1)

# Add the regular-special edges:
for v in packing.verts:
    p = dualPoints[v.index]
    for f in v.faces():
        q = dualPtCP(f.data)
        seg = SegmentE3(p, q)
        viewer.add(seg)
        viewer.setStyle(seg, purpleStroke1)




viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=800, objects='[[{"type": "DiskS2", "disk": [-6.179676972449112, -9.282032302755095, 5.35898384…

In [25]:
random.random()

0.8539750173479087